In [1]:
import torch

Data Preprocessing


In [2]:
import os
import sys
import h5py
import torch
import numpy as np
from src.data import Data
from src.utils.color import to_float_rgb

# Set your file path for imports
file_path = os.path.dirname(os.path.abspath(''))  # for .ipynb notebook
sys.path.append(file_path)

def read_custom_tile(filepath, xyz=True, rgb=True, intensity=True, semantic=True, remap=True):
    """
    Read a custom dataset saved as .h5 and return the data in a structured format.

    :param filepath: str
        Absolute path to the .h5 file.
    :param xyz: bool
        Whether XYZ coordinates should be saved in the output Data.pos.
    :param rgb: bool
        Whether RGB colors should be saved in the output Data.rgb.
    :param intensity: bool
        Whether intensity should be saved in the output Data.intensity.
    :param semantic: bool
        Whether semantic labels should be saved in the output Data.y.
    :param remap: bool
        Whether to remap labels to a trainable format (e.g., reassigning label indices).
    """
    # Create an empty Data object
    data = Data()

    # Read the .h5 file
    with h5py.File(filepath, 'r') as f:
        pointcloud = f['pointcloud'][:]
        # pointcloud = np.array(f['pointcloud'], dtype=np.float32)
    
    # Populate data with point coordinates (XYZ)
    if xyz:
        data.pos = torch.tensor(pointcloud[:, :3], dtype=torch.float32)

    # Populate data with point RGB colors
    if rgb:
        data.rgb = to_float_rgb(torch.tensor(pointcloud[:, 4:7], dtype=torch.float32))

    # Populate data with point LiDAR intensity
    if intensity:
        data.intensity = torch.tensor(pointcloud[:, 3], dtype=torch.float32)  # 'i' for intensity

    # Populate data with point semantic segmentation labels
    if semantic:
        labels = pointcloud[:, 7]  # 'l' stands for label
        data.y = torch.LongTensor(labels)
        if remap:
            data.y = torch.from_numpy(ID2TRAINID)[data.y]

    return data

# def read_custom_tile(filepath, xyz=True, rgb=True, intensity=True, semantic=True, remap=True):
#     """
#     Read a .pts/txt file and return the data in a structured format.

#     :param filepath: str
#         Absolute path to the .pts file.
#     :param xyz: bool
#         Whether XYZ coordinates should be saved in the output Data.pos.
#     :param rgb: bool
#         Whether RGB colors should be saved in the output Data.rgb.
#     :param intensity: bool
#         Whether intensity should be saved in the output Data.intensity.
#     :param semantic: bool
#         Whether semantic labels should be saved in the output Data.y.
#     :param remap: bool
#         Whether to remap labels to a trainable format (e.g., reassigning label indices).
#     :return: Data
#         A structured data object with the desired attributes.
#     """
#     # Create an empty Data object
#     data = Data()

#     # Read the .pts file
#     pointcloud = np.loadtxt(filepath)

#     # Populate data with point coordinates (XYZ)
#     if xyz:
#         data.pos = torch.tensor(pointcloud[:, :3], dtype=torch.float32)

#     # Populate data with point RGB colors
#     if rgb and pointcloud.shape[1] >= 6:
#         data.rgb = to_float_rgb(torch.tensor(pointcloud[:, 3:6], dtype=torch.float32))

#     # Populate data with point LiDAR intensity
#     if intensity and pointcloud.shape[1] >= 7:
#         data.intensity = torch.tensor(pointcloud[:, 6], dtype=torch.float32)

#     # Populate data with point semantic segmentation labels
#     if semantic and pointcloud.shape[1] >= 8:
#         labels = pointcloud[:, 7]  # 'l' stands for label
#         data.y = torch.LongTensor(labels)
#         if remap:
#             data.y = torch.from_numpy(ID2TRAINID)[data.y]

#     return data

# Number of classes in your dataset (excluding void/unlabeled/ignored)
NUM_CLASSES = 3

# Mapping from original classes to new training class IDs (if remapping is necessary)
ID2TRAINID = np.asarray([
    # NUM_CLASSES,  # 0 -> Ignored
    0,            # 1 -> Class 0 (e.g., 'Window')
    1,          # 2 -> Class 1 (e.g., 'Wall')
    2             # 3 -> Class 2 (e.g., 'Door')
])

# Class names (including void/unlabeled/ignored last)
# CLASS_NAMES = [
#     'Window',
#     'Wall',
#     'Door',
#     # 'Ignored'
# ]

CLASS_NAMES = [
    'Window + doors',
    'Wall',
    'Others',
    # 'Ignored'
]

# Class color palette (including void/unlabeled/ignored last)
CLASS_COLORS = np.asarray([
    [255, 255, 0],    # Yellow for 'Window'
    [0, 255, 0],      # Green for 'Wall'
    [0, 0, 255]      # Blue for 'Others'
    # [0, 0, 0]         # Black for 'Ignored'
])


In [ ]:
# !pwd
filepath = '/mnt/e/pointnet2_pytorch_semantic/data/s3dis/buildings_3_labels_original/FRP2_SE.h5'#B4020_SE.h5' #FRP2_NW.h5'
# filepath = 'data/building/raw/test/KCDC111-112_S_processed.h5'# FRP2_NW.h5'KCDC111-112_W_processed.h5'
# filepath = '/mnt/e/debbie/FRP2_october_trimmed.txt'
# filepath = "/mnt/e/superpoint_transformer/NRC_DATA/all/NRCAN_cropped.txt"
# filepath ="NRC_DATA/all/NRCAN_cropped.txt" #KCDC_dummy.txt" 
data = read_custom_tile(filepath)
# print(data.shape)
data


In [ ]:
print(f"Data positions: {data.pos.shape}")

In [ ]:
data.num_points

In [ ]:
data.keys

In [7]:
# data.show(class_colors = CLASS_COLORS,class_names = CLASS_NAMES, max_points=1000000)

In [8]:
# data.show(center=[4.25, 2.82, 1.5], radius=30, keys=['intensity'], class_names=CLASS_NAMES, class_colors=CLASS_COLORS)


In [9]:
# from src.transforms import SampleXYTiling, GridSampling3D
# from src.data import Batch

# # Tile the cloud into `xy_tiling` XY-oriented chunks of equal horizontal 
# # span
# xy_tiling = (1, 1)

# # Voxelize the point cloud only for the sake of faster computation and 
# # visualization here
# # data_5m = GridSampling3D(0.009)(data)
# data_5m = data

# # Compute each chunk 
# chunks = []
# for x in range(xy_tiling[0]):
#     for y in range(xy_tiling[1]):        
#         # Extract the chunk at (x, y) in the tiling grid
#         chunk = SampleXYTiling(x=x, y=y, tiling=xy_tiling)(data_5m)

#         # Add a 'tile' attribute to the points for visualization
#         chunk.tile = torch.full((chunk.num_points,), x * xy_tiling[1] + y)
        
#         # Store the chunk for later aggregation
#         chunks.append(chunk)

# # Aggregate all chunk `Data` objects into one big `Data` object
# data_tiled = Batch.from_data_list(chunks)

# # Show the resulting `Data' with the 'tile' attribute
# data_tiled.show(keys='tile')

In [10]:
# from src.transforms import SampleRecursiveMainXYAxisTiling, GridSampling3D
# from src.data import Batch

# # Recursively tile the cloud into `2**pc_tiling` chunks with respect to 
# # principal components of the XY coordiantes
# pc_tiling = 2

# # Voxelize the point cloud only for the sake of faster computation and 
# # visualization here
# # data_5m = GridSampling3D(0.009)(data)
# data_5m = data
# # Compute each chunk 
# chunks = []
# for x in range(2**pc_tiling):
#     # Extract the chunk at x in the recursive tiling
#     chunk = SampleRecursiveMainXYAxisTiling(x=x, steps=pc_tiling)(data_5m)

#     # Add a 'tile' attribute to the points for visualization
#     chunk.tile = torch.full((chunk.num_points,), x)
    
#     # Store the chunk for later aggregation
#     chunks.append(chunk)

# # Aggregate all chunk `Data` objects into one big `Data` object
# data_tiled = Batch.from_data_list(chunks)

# # Show the resulting `Data' with the 'tile' attribute
# data_tiled.show(keys='tile')

In [11]:
# from src.transforms import SampleXYTiling

# # Extract the chunk at (x, y) in the tiling grid
# data = SampleXYTiling(x=0, y=0, tiling=4)(data)

In [12]:
from src.utils import init_config

# cfg = init_config(overrides=[f"experiment=semantic/dales"])
cfg = init_config(overrides=[f"experiment=semantic/building"])


In [ ]:
cfg.keys()

In [14]:
# cfg = init_config(overrides=[f"experiment=semantic/s3dis"])
# cfg.keys()

In [ ]:
from src.transforms import instantiate_datamodule_transforms

transforms_dict = instantiate_datamodule_transforms(cfg.datamodule)
transforms_dict

In [ ]:
# print(f"Shape of coords: {coords.shape}")
print(f"Data positions: {data.pos.shape}")

In [17]:
# Apply pre-transforms
nag = transforms_dict['pre_transform'](data)

# Simulate the behavior of the dataset's I/O behavior with only
# `point_load_keys` and `segment_load_keys` loaded from disk
from src.transforms import NAGRemoveKeys
nag = NAGRemoveKeys(level=0, keys=[k for k in nag[0].keys if k not in cfg.datamodule.point_load_keys])(nag)
nag = NAGRemoveKeys(level='1+', keys=[k for k in nag[1].keys if k not in cfg.datamodule.segment_load_keys])(nag)

# Move to device
nag = nag.cuda()

# Apply on-device transforms
nag = transforms_dict['on_device_test_transform'](nag)

In [ ]:
nag

In [19]:
# nag.show(class_names=CLASS_NAMES, class_colors=CLASS_COLORS, center=[485, 505, 0], radius=0.02, keys=nag[0].keys, centroids=True, h_edge=True)
# nag.show(class_names=CLASS_NAMES, class_colors=CLASS_COLORS,radius =20, keys=nag[0].keys, centroids=True, h_edge=True)

In [20]:
import hydra 
from src.utils import init_config

# Path to the checkpoint file downloaded from https://zenodo.org/records/8042712
# ckpt_path = "/path/to/your/superpoint_transformer.ckpt"
# ckpt_path = "/mnt/e/superpoint_transformer/logs/train/runs/2024-10-28_10-05-50/checkpoints/epoch_959.ckpt"
# ckpt_path = "/mnt/e/superpoint_transformer/logs/train/runs/2024-10-23_16-02-51/checkpoints/epoch_1659.ckpt"
# ckpt_path = "/mnt/e/superpoint_transformer/logs/train/runs/2024-10-26_13-04-21/checkpoints/epoch_1789.ckpt"
# ckpt_path = "/mnt/e/superpoint_transformer/logs/train/runs/2024-10-28_11-28-29/checkpoints/epoch_1889.ckpt"
# ckpt_path = "/mnt/e/superpoint_transformer/logs/train/runs/2024-10-30_10-37-41/checkpoints/epoch_849.ckpt"  # 2 labels 1000 epochs
ckpt_path = "/mnt/e/superpoint_transformer/logs/train/runs/2024-11-13_11-05-27/checkpoints/epoch_879.ckpt" # 2 labels 1000 epochs new parameters 

# cfg = init_config(overrides=[f"experiment=semantic/s3dis"])
cfg = init_config(overrides=[f"experiment=semantic/building"])

# Instantiate the model and load pretrained weights
model = hydra.utils.instantiate(cfg.model)
model = model._load_from_checkpoint(ckpt_path)

In [21]:
# Set the model in inference mode on the same device as the input
model = model.eval().to(nag.device)

# Inference, returns a task-specific ouput object carrying predictions
with torch.no_grad():
    output = model(nag)

In [ ]:
output.semantic_pred.shape, nag.num_points

In [23]:
# Compute the level-0 (voxel-wise) semantic segmentation predictions 
# based on the predictions on level-1 superpoints and save those for 
# visualization in the level-0 Data under the 'semantic_pred' attribute

nag[0].semantic_pred = output.voxel_semantic_pred(super_index=nag[0].super_index)

In [ ]:
# from src.datasets.dales import CLASS_NAMES as DALES_CLASS_NAMES
# from src.datasets.dales import CLASS_COLORS as DALES_CLASS_COLORS

# from src.datasets.s3dis import CLASS_NAMES 
# from src.datasets.s3dis import CLASS_COLORS
# from src.datasets.building import CLASS_NAMES 
# from src.datasets.building import CLASS_COLORS

CLASS_NAMES = [
    'Window + Door',
    'Wall',
    'Others'
]

CLASS_COLORS = np.asarray([
    [255, 255, 0],  # Class 0: Yellow for 'Window + Door'
    [0, 255, 0],    # Class 1: Green for 'Wall'
    [0, 0, 255]     # Class 2: Blue for 'Others'
])

# nag.show(class_names=CLASS_NAMES, class_colors=CLASS_COLORS)#, radius=10)
nag.show(class_names=CLASS_NAMES,class_colors=CLASS_COLORS,max_points=500000,centroids=False,v_edge=True, h_edge=True)#,title="KCDC_visualization",path="KCDC_visualization.html")#,gap=[4, 4, 0])
# nag.show(class_names=CLASS_NAMES,class_colors=CLASS_COLORS,stuff_classes=dataset.stuff_classes,num_classes=dataset.num_classes,max_points=100000)

In [ ]:
# Ratio of sizes of successive partition levels
nag.level_ratios

In [ ]:
print(nag[2].semantic_segmentation_oracle(NUM_CLASSES))

In [ ]:
# Oracle semantic segmentation metrics on P_1
print(nag[1].semantic_segmentation_oracle(NUM_CLASSES))

In [ ]:
# Oracle semantic segmentation metrics on P_0
print(nag[0].semantic_segmentation_oracle(NUM_CLASSES))

In [ ]:
transforms_dict['pre_transform']